In [1]:
import numpy as np
import json


def get_specs():
    '''
    Reads the species file, using code from ChemTorch.
        chemtype: 'C' for carbon-rich,  'O' for oxygen-rich
        rate: indicates the used rate file. 16 --> Rate16, UMIST

    Returns a dictionary with the species and their index, 
        and a dictionary with the index and the species.

    '''



    loc_specs = '/STER/silkem/fortran_0D/rates/rate16.specs'
    
    specs = np.loadtxt(loc_specs, usecols=(1), dtype=str,  max_rows=469)  
    mass = np.loadtxt(loc_specs, usecols=(2), dtype=float,  max_rows=469)  

    specs_dict = dict()
    idx_specs  = dict()
    for i in range(len(specs)):
        specs_dict[specs[i]] = i
        idx_specs[i] = specs[i]

    return specs_dict, idx_specs, mass

def conversion_to_krome():
    conv_dict = get_conversion()

    krome_idx = np.linspace(0,len(conv_dict),len(conv_dict)).astype('int')
    for ikey, key in enumerate(conv_dict):
        # print(type(int(conv_dict[key][1])), conv_dict[key][1])
        krome_idx[ikey] = int(conv_dict[key][1])-1
        # break

    umist_idx = np.linspace(1,len(conv_dict),len(conv_dict)).astype('int')
    
    return krome_idx, umist_idx

def get_conversion():
    with open('networks/specs_UMISTrate16_to_krome.json', 'r') as f:
        conv=f.read()
    conv_dict  = json.loads(conv)

    return conv_dict

def sort_arrays(arr1, arr2):
    # Combine the two arrays into a list of tuples
    combined = list(zip(arr1, arr2))

    # Sort the list of tuples
    combined.sort()

    # Split the sorted list of tuples back into two arrays
    sorted_arr1, sorted_arr2 = zip(*combined)

    return list(sorted_arr1), list(sorted_arr2)



specs_dict, idx_specs, mass = get_specs()

krome_idx, umist_idx = conversion_to_krome()
sorted_krome, sorted_umist = sort_arrays(krome_idx,umist_idx)

In [2]:
krome_specs = list()
path ='/STER/silkem/fortran_0D/rates/rate16_krome.specs'


with open(path, 'w') as f:
    for i in range(len(krome_idx)):
        print(i+1, idx_specs[sorted_umist[i]-1])
        krome_specs.append(idx_specs[sorted_umist[i]-1])
        # f.write(f'{i+1}     {idx_specs[sorted_umist[i]-1]}  {mass[sorted_umist[i]-1]}\n')
# for i in range(len(umist_idx)):
#     krome_specs.append(idx_specs[sorted_umist[i]-1])



1 C-
2 e-
3 C2-
4 C3-
5 C4-
6 C5-
7 C6-
8 C7-
9 C8-
10 C10-
11 C2H-
12 C3H-
13 C3N-
14 C4H-
15 C5H-
16 C6H-
17 C7H-
18 C8H-
19 C9-
20 C9H-
21 CH-
22 O-
23 OH-
24 S-
25 CN-
26 H-
27 C10H-
28 C5N-
29 O2-
30 C
31 C2
32 CH2
33 C2H2
34 CH
35 C2H
36 CO2
37 CO
38 H2O
39 H2CO
40 N
41 CN
42 NH
43 HCN
44 O2
45 O
46 OH
47 HCO
48 C4
49 C3
50 C5
51 C6
52 C7
53 C8
54 C9
55 C10
56 C3H
57 C4H
58 C4N
59 C5H
60 C6H
61 C7H
62 C8H
63 C9H
64 C10H
65 CS
66 CH3
67 CH3CN
68 CH3OH
69 OCS
70 CH4
71 H
72 H2
73 NH2
74 NH3
75 C10H2
76 H2CCC
77 HC3N
78 HC4H
79 C5H2
80 HC5N
81 C6H2
82 C7H2
83 C8H2
84 C9H2
85 HS
86 C2N
87 C3N
88 C5N
89 C7N
90 HC7N
91 C9N
92 HC9N
93 NO
94 NS
95 NO2
96 SO2
97 C2O
98 SO
99 HNC
100 C2H4
101 C2H5
102 C2H5OH
103 C2S
104 C3O
105 C3S
106 C4H3
107 C4S
108 C6H6
109 CCP
110 CCl
111 CH2CCH2
112 CH2CCH
113 CH2CN
114 CH2CO
115 CH3CCH
116 CH3CHCH2
117 CH3CHO
118 CH3COCH3
119 CH3OCH3
120 CP
121 ClO
122 Fe
123 H2S
124 H2SiO
125 HCOOCH3
126 HCP
127 HPO
128 Mg
129 NCCN
130 Na
131 P
132 PH
133 PO
134 Si

In [6]:
def read_input(infile):

    # with open(infile, 'a') as file:
    #         file.write('\nName = '+name+'\n')

    with open(infile,'r') as f:
        file = f.readlines()
        lines = []
        for line in file:
            lines.append(line.split())

    inputfile = {}
    for i in range(len(lines)):
        if not len(lines[i]) == 0 and len(lines[i]) > 2:
            inputfile[lines[i][0]] = lines[i][2]
        elif not len(lines[i]) == 0 and len(lines[i]) <= 2:
            print('You forgot to give an input for '+lines[i][0])

    return inputfile

def read_abs(infile):
    abs = np.loadtxt(infile, usecols = (1))

    return abs

def convert_specs_umist_to_krome1(n_umist, krome_idx):

    n_krome = np.zeros(len(n_umist))

    for i in range(len(n_umist)):
        # print(i, krome_idx[i])
        n_krome[krome_idx[i]] = n_umist[i]

    return n_krome

def write_abs(loc, path, i, abs):
    # print(len(idx_specs), len(abs))
    # with open(f"{loc}out/{path}/time.out", "w") as file:
    #     file.write('')
    with open(f"{loc}rates/{path}/krome/abs_{i}.txt", "w") as file:
        for j in range(len(abs)):
            # print(j, sorted_umist[j])
            # if abs[j] != 0.:
            #     print(f"{idx_specs[sorted_umist[j]-1]} {abs[j]}")
            line = '{:>3}  {:<12}   {:<12}'.format(j+1, idx_specs[sorted_umist[j]-1], abs[j])
            file.write(line)
            file.write('\n')




In [4]:
path0D = '/STER/silkem/fortran_0D/'

mod = '20211015_gridC_Mdot1e-6_v17-5_T_eps_model_2022-12-24h17-06-51/'


In [5]:
# print(krome_idx)


for i in range(1,134):
    # print(i)
    path = path0D + 'rates/' + mod 
    n_umist = read_abs(path + f'CSEcode/abs_{i}.txt')
    # print(len(n_umist))
    
    n_krome = convert_specs_umist_to_krome1(n_umist, krome_idx)

    write_abs(path0D, mod, i, n_krome)

    



In [6]:
## In UMIST kations are indicated with a '+', anions with a '-'.
## In KROME kations are indicated with a 'j', anions with a 'k'.


converge_UMISTtoKROME = { 'H' : [1,71],
                          'H+': [2,202],
                        'H-': [3,26],                   
                    'H2+': [4,336],
                    'H3+': [5,406],
                    'He': [6,173],
                    'He+': [7,338],
                    'HeH+': [8,441],
                    'C-': [9,1],
                    'C+': [10,204],
                    'C': [11,30],     
                    'CH-': [12,21],  
                    'CH+': [13,227],             
                    'CH': [14,34],
                    'CH2+': [15,218],
                    'N+': [16,289],
                    'CH2': [17,32],
                    'N': [18,40],
                    'CH3': [19,66],
                    'NH': [20,42],
                    'NH+': [21,326],
                    'CH3+': [22,286],                          
                    'O+': [23,284],
                    'NH2+': [24,283],
                    'CH4+': [25,287],                                        
                    'O-': [26,22],                    
                    'NH2': [27,73],                    
                    'O': [28,45],
                    'CH4': [29,70],
                    'OH-': [30,23],
                    'OH': [31,46],
                    'OH+': [32,285],
                    'CH5+': [32,393],
                    'NH3+': [34,241],                    
                    'NH3': [35,74],
                    'NH4+': [36,446],                   
                    'H2O': [37,38],
                    'H2O+': [38,282],
                    'F': [39,172],
                    'H3O+': [40,413],
                    'F+': [41,337],                  
                    'HF': [42,194],
                    'HF+': [43,430],                    
                    'H2F+': [44,400],
                    'Na': [45,130],
                    'Na+': [46,244],
                    'C2-': [47,3],
                    'C2+': [48,261],
                    'Mg': [49,128],
                    'Mg+': [50,238],
                    'C2': [51,31],                 
                    'C2H-': [52,11],
                    'C2H+': [53,267],
                    'C2H': [54,35],
                    'CN': [55,41],
                    'C2H2': [56,33],
                    'C2H2+': [57,268],
                    'CN-': [58,25],
                    'CN+': [59,263],
                    'HCN': [60,43],
                    'C2H3': [61,147],                    
                    'C2H3+': [62,269],                    
                    'HCN+': [63,276],
                    'HNC': [64,99],
                    'N2': [65,146 ],
                    'C2H4+': [66,206],
                    'Si+': [67,250],
                    'H2CN': [68,191],
                    'H2NC+': [69,401],
                    'N2+': [70,265],
                    'Si': [71,134],
                    'CO': [72,37],
                    'CO+': [73,264],
                    'C2H4': [74,100],
                    'HCNH+': [75,426],
                    'C2H5+': [76,207],
                    'HCO': [77,47],
                    'SiH+': [78,333],
                    'HCO+': [79,201],
                    'CH2NH': [80,185],
                    'SiH': [81,169],
                    'C2H5': [82,101],
                    'HOC+': [83,203],
                    'N2H+': [84,442],
                    'CH2NH2+': [85,380],
                    'SiH2': [86,141],                  
                    'CH3CH3+': [87,280],
                    'NO': [88,93],
                    'H2CO': [89,39],
                    'CH4N+': [90,392],
                    'H2CO+': [91,231],
                    'CH3CH3': [92,148],
                    'SiH2+': [93,257],
                    'NO+': [94,242],
                    'P+': [95,246],
                    'P': [96,131],
                    'H3CO+': [97,411],
                    'HNO+': [98,343],
                    'SiH3+': [99,258],                    
                    'HNO': [100,177],
                    'CF+': [101,378],
                    'C2H7+': [102,352],                    
                    'SiH3': [103,142],    
                    'O2': [104,44],                    
                    'SiH4': [105,168],
                    'PH': [106,132],
                    'S':    [107,145],
                    'H2NO+': [108,403],
                    'CH3OH': [109,68],
                    'SiH4+': [110,332],
                    'O2+': [111,266],
                    'S-': [112,24],
                    'O2-': [113,29],
                    'PH+': [114,247],
                    'CH3OH+': [115,288],                    
                    'S+': [116,262],
                    'CH3OH2+': [117,391],
                    'O2H': [118,198],
                    'PH2+': [119,327],
                    'HS': [120,85],
                    'HS+': [121,324],                    
                    'PH2': [122,163],                  
                    'O2H+': [123,447],
                    'SiH5+': [124,465],                    
                    'H2O2': [125,192],
                    'H2S+': [126,232],
                    'H2S': [127,123],
                    'PH3+': [128,457],                   
                    'H3S+': [129,414],
                    'Cl': [130,150],
                    'Cl+': [131,292],
                    'C3+': [132,278],
                    'HCl': [133,159],
                    'C3-': [134,4],
                    'C3': [135,49],                  
                    'HCl+': [136,321],   
                    'H2Cl+': [137,399],                 
                    'C3H-': [138,12],
                    'C3H+': [139,296],
                    'C3H': [140,56],
                    'C2N': [141,86],
                    'C3H2+': [142,295],
                    'C2N+': [143,239],
                    'H2CCC': [144,76],
                    'CNC+': [145,240],
                    'C3H2': [146,151],
                    'C2NH+': [147,353],
                    'C3H3+': [158,273],
                    'CH2CCH': [149,112],
                    'CH2CCH+': [150,220],                  
                    'SiC': [151,138],
                    'CH2CCH2': [152,111],
                    'SiC+': [153,254],
                    'C2O': [154,97],
                    'C3H4+': [155,219],
                    'C2O+': [156,209],
                    'CH2CN': [157,113],
                    'CH3CCH': [158,115],
                    'CH2CN+': [159,221],
                    'C3H5+': [160,355],
                    'HCSi': [161,158],
                    'HCSi+': [162,320],
                    'CH3CN+': [163,313],
                    'HC2O+': [164,418],                    
                    'CH3CN': [165,67],
                    'C3H6+': [166,223],
                    'SiN': [167,143],
                    'CH3CNH+': [168,386],
                    'NH2CN': [169,197],
                    'CH3CHCH2': [170,116],
                    'SiN+': [171,259],                    
                    'SiCH2+': [172,255],
                    'SiCH2': [173,139],
                    'CNO': [174,190],
                    'CH2CO+': [175,222], 
                    'OCN': [176,199],
                    'OCN+': [177,448],                   
                    'CH2CO': [178,114],                    
                    'CP+': [179,228],
                    'HNCO': [180,175],
                    'HCNO+': [181,340],
                    'HNCO+': [182,341],
                    'HONC': [183,176],
                    'SiCH3+': [184,256],
                    'HCNO': [185,174],
                    'CP': [186,120],
                    'C3H7+': [187,356],                    
                    'NH2CNH+': [188,445],
                    'HNSi+': [189,322],
                    'HOCN': [190,196],                    
                    'HNSi': [191,160],
                    'CH3CO+': [192,387],                    
                    'HOCN+': [193,434],
                    'SiCH3': [194,140],
                    'HONC+': [195,342],
                    'CS+': [196,291],
                    'HCNOH+': [197,427],
                    'N2O+': [198,325],
                    'CS': [199,65],
                    'SiCH4+': [200,463],
                    'CO2+': [201,274],                    
                    'H2CNO+': [202,398],
                    'HNCOH+': [203,432],
                    'SiNH2+': [204,467],
                    'HCP': [205,126], 
                    'SiO+': [206,335],
                    'CH3CHO': [207,117], 
                    'SiO': [208,171], 
                    'H2NCO+': [209,402],
                    'CH3CHO+': [210,224],
                    'N2O': [211,162],
                    'H2OCN+': [212,404],
                    'HCP+': [213,236],                    
                    'CO2': [214,36],
                    'CH3CHOH+': [215,385],
                    'HCS+': [216,347],                    
                    'SiOH+': [217,468],
                    'PN+': [218,281],
                    'PN': [219,149],
                    'HCS': [220,193],
                    'PCH2+': [221,454],
                    'HN2O+': [222,431],                    
                    'HCO2+': [223,428],                  
                    'NS+': [224,243],
                    'H2SiO': [225,124],
                    'H2CS': [226,155],
                    'NS': [227,94],                    
                    'CH2PH': [228,186],
                    'HCOOH+': [229,344],
                    'C2H5OH+': [230,208],
                    'NO2':  [231,95],
                    'CH3OCH3': [232,119],
                    'H2CS+': [233,314],
                    'HCOOH': [234,178],
                    'PCH3+': [235,455],
                    'H2SiO+': [236,233],
                    'CH3OCH3+': [237,226],
                    'C2H5OH': [238,102],
                    'NO2+': [239,345],                  
                    'HPN+': [240,436],                    
                    'CH3OCH4+': [241,390],
                    'SiF+': [242,464],
                    'PNH2+': [243,458],
                    'CCl': [244,110],
                    'H3CS+': [245,412],
                    'PO+': [246,248],
                    'CCl+': [247,217],
                    'HNS+': [248,433],
                    'PCH4+': [249,456],
                    'HCOOH2+': [250,429],
                    'PO': [251,133],
                    'C2H5OH2+': [252,351],
                    'H3SiO+': [253,416],
                    'C4-': [254,5],
                    'HPO+': [255,237],
                    'HPO': [256,127],
                    'C4': [257,48],
                    'SO': [258,98],
                    'C4+': [259,297],
                    'SO+': [260,249],
                    'PNH3+': [261,459],                    
                    'C4H+': [262,298],
                    'H2CCl+': [263,397],
                    'H2PO+':[264,405],
                    'C4H': [265, 57],
                    'C4H-': [266,14],
                    'HSO+': [267,437],
                    'C4H2+': [268,234],                    
                    'C3N': [269, 87],
                    'HC4H':[270,78],
                    'C3N+':[271,339],                    
                    'C3N-': [272,13],
                    'HC3N+': [273,275],
                    'C4H3': [274,106],
                    'HC3N': [275,77],
                    'ClO' : [276,121],
                    'ClO+' : [277,229],
                    'C4H3+': [278,213],                   
                    'HNC3': [279,195],
                    'C3O': [280,104],
                    'CH2CHCCH': [281,182],
                    'SiC2': [282,135],
                    'SiC2+': [283,251],
                    'NCCN' : [284,129],
                    'HC3NH+': [285,420],
                    'C4H4+': [286,357],
                    'C2N2+': [287,277],
                    'C3O+': [288,211],
                    'HC3O+': [289, 421],
                    'C4H5+': [290, 358],                    
                    'SiC2H': [291,136],
                    'CH2CHCN' : [292,184 ],
                    'CH2CHCN+': [293,379],
                    'SiC2H+': [294, 252],
                    'NCCNH+': [295,444],
                    'SiC2H2+': [296,329],
                    'CH2CHCNH+': [297, 346],
                    'CH2CHCHCH2': [298,183],
                    'C3H2O+': [299, 354],
                    'SiC2H2' : [300, 165],
                    'SiNC+': [301, 334],
                    'SiNC' : [302, 170],  
                    'SiNCH+' : [303,466],
                    'CCP' : [304, 109], 
                    'C2H5CN' :[305, 180], 
                    'SiC2H3+' : [306,460 ],
                    'CCP+' : [307, 216],
                    'C4H7+' : [308,359],
                    'H3C3O+' : [309, 407],
                    'HC2P' : [310, 157],
                    'HC2P+' : [311, 316],                    
                    'Fe+' : [312, 230],
                    'C2H5CNH+': [313,350 ],
                    'C2S' : [314, 103],
                    'C2S+' : [315, 210],
                    'Fe' : [316, 122],
                    'HC2S+' : [317, 419],                    
                    'PC2H2+': [318, 449],
                    'CH3COCH3+': [319, 225],
                    'CH3COCH3': [320, 118],
                    'PC2H3+': [321, 450],
                    'CH3COCH4+' : [322, 388],                    
                    'CH3CS+': [323, 389],                     
                    'PC2H4+' : [324,451],
                    'HCOOCH3' : [325,125],
                    'C5': [326, 50],
                    'OCS+': [327, 245],
                    "SiO2": [328, 200],
                    'C5-' : [329, 6],                    
                    'SiS+' : [330, 260],  
                    'OCS' : [331, 69],
                    'SiS': [332, 144],
                    'C5+' : [333, 279],                        
                    "COOCH4+": [334, 235],
                    'HOCS+': [335, 435],
                    "HSiS+": [336, 440],
                    "C5H+": [337, 300],
                    "HSiO2+": [338, 439],
                    'C5H': [339, 59],
                    'H5C2O2+': [340, 417],                    
                    'C5H-': [341, 15],
                    'C5H2+': [342, 270],
                    "C4N+": [343, 360],
                    'C5H2': [344, 79],
                    'C4N': [345, 58],
                    'HC4N+': [346, 423],                    
                    'C5H3+': [347, 361],
                    'CH3C4H': [348, 153],
                    'SO2': [349, 96],
                    'S2': [350, 164], 
                    'SiC3+': [351, 253],
                    'CH3C4H+': [352, 311],
                    'S2+': [353, 328],
                    'SiC3': [354, 137],                   
                    'H2C4N+': [354, 394],
                    'SO2+': [356, 290],
                    'HS2': [357, 161],
                    'HSO2+': [358, 438],
                    'SiC3H+': [359, 330],
                    "CH3C3N+": [360, 381],
                    'HS2+': [361, 323],
                    'CH3C3N': [362, 187],
                    'SiC3H': [363, 166],                   
                    'C5H5+': [364, 362],
                    'CH3C3NH+': [365, 382],
                    'H2S2+': [366, 315],
                    'SiC3H2+': [367, 461],
                    'H2S2': [368, 156],
                    'H3S2+': [369, 415],                    
                    'NCCNCH3+': [370, 443],
                    'C3P': [371, 181],
                    'PC3H+': [372, 452],                    
                    'C3S': [373, 105],
                    'C3S+': [374, 212],                    
                    'HC3S+': [375, 422],
                    'C6-': [376, 7],
                    'C6+': [377, 301],                    
                    'C6': [378, 51],
                    'C6H': [379, 60],
                    'C6H+': [380, 302],                    
                    'C6H-': [381, 16],                    
                    'C6H2': [382, 81],
                    'C6H2+': [383, 271],
                    'C5N': [384, 88],
                    'C5N-': [385, 28],
                    'C5N+': [386, 363],
                    'HC5N+': [387, 317],                       
                    'HC5N': [388, 80],    
                    'C6H3+': [389, 364],    
                    'SiC4+': [390, 331],
                    'C6H4+': [391, 365],
                    'HC5NH+': [392, 425],
                    'SiC4': [393, 167],
                    'H3C5N+': [395, 408],
                    'SiC4H+': [394, 462],                    
                    'C6H5+': [396, 366],
                    'C6H6': [397, 108],
                    'C6H6+': [398, 215],
                    'C4P': [399, 152],
                    'C6H7+': [400, 367],                    
                    'C4P+': [401, 299],
                    'C4S': [402, 107],
                    'PC4H+': [403, 453],
                    'C4S+': [404, 214],                    
                    'HC4S+': [405, 424],
                    'C7+': [406, 303],
                    'C7': [407, 52],
                    'C7-': [408, 8],                    
                    'C7H-': [409, 17],
                    'C7H+': [410, 304],
                    'C7H': [411, 61],
                    'C7H2+': [412, 272],
                    'C7H2': [413, 82],
                    'C7H3+': [414, 368],
                    'C7H4+': [415, 312],
                    'CH3C6H': [416, 154],
                    'CH3C5N': [417, 188],
                    'C7H5+': [418, 369],
                    'CH3C5NH+': [419, 383],
                    'C8-': [420, 9],
                    'C8': [421, 53],
                    'C8+': [422, 305],   
                    'C8H': [423, 62],                 
                    'C8H+': [424, 307],                    
                    'C8H-': [425, 18],
                    'C8H2': [426, 83],
                    'C7N': [427, 89],
                    'C7N+': [428, 370],                    
                    'C8H2+': [429, 306],
                    'HC7N': [430, 90],
                    'HC7N+': [431, 318],
                    'C8H3+': [432, 371],                    
                    'C8H4+': [433, 372],
                    'H2C7N+': [434, 395],         
                    'C8H5+': [435, 373],   
                    'H3C7N+': [436, 409],
                    'C9': [437, 54],
                    'C9+': [438, 308],
                    'C9-': [439, 19],
                    'C9H+': [440, 310],
                    'C9H-': [441, 20],
                    'C9H': [442, 63], 
                    'C9H2+': [443, 309],                   
                    'C9H2': [444, 84],                    
                    'C9H3+': [445, 374],
                    'C9H4+': [446, 375],
                    'CH3C7N': [447, 189],
                    'C9H5+': [448, 376],                    
                    'CH3C7NH+': [449, 384],
                    'C10-': [450, 10],
                    'C10': [451, 55],
                    'C10+': [450, 293],
                    'C10H+': [455, 205],
                    'C10H-': [454, 27],
                    'C10H': [453, 64],
                    'C9N': [458, 91],
                    'C9N+': [457, 377],
                    'C10H2': [456, 75],
                    'C10H2+': [459, 294],
                    'HC9N+': [460, 319],
                    'C10H3+': [462, 348],
                    'HC9N': [461, 92],                    
                    'H2C9N+': [463, 396],
                    'H3C9N+': [464, 410],
                    'C11': [466, 179],
                    'C11+': [465, 349],
                    'e-': [467, 2],
                    'H2': [468, 72]
}

import json

json_object = json.dumps(converge_UMISTtoKROME, indent=4)
with open("networks/specs_UMISTrate16_to_krome.json", "w") as outfile:
    outfile.write(json_object)